<a href="https://colab.research.google.com/github/Geshanth/AirPassengerReviews/blob/main/CSI4142_D2_G30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSI4142 Group 30**

**DATA STAGING**

Connecting to our PostgreSQL database which runs locally! to run this notebook you must have PostgresSQL installed and also install the necessary libraries. If making the database connection is impossible, all the cells still work except where a connection is needed.

In [ ]:
print("The notebook is running")

The notebook is running


In [ ]:
!pip install psycopg2-binary

In [ ]:
import psycopg2

try:
    connection = psycopg2.connect(
        host="localhost",  # or use 127.0.0.1
        port="5433",  # Make sure this matches your PostgreSQL configuration
        database="csi4142",  # Your database name
        user="postgres",  # Your database user
        password=""  # Use Your own database password
    )
    cursor = connection.cursor()  #cursor is the connection object
    # If connection is successful, you can proceed to execute queries using connection.cursor()
    print("Connection established successfully!")
except Exception as e:
    print("Error connecting to the database:", e)


Error connecting to the database: connection to server at "localhost" (::1), port 5433 failed: fe_sendauth: no password supplied



In [ ]:
import pandas as pd
import numpy as np

**Loading the main Dataframe(base cube)**

In [ ]:
#two main dataframe to implement some functionalities
url= 'https://raw.githubusercontent.com/Sattar-A/CSI4142_G30/main/Datasets/Electric_Vehicle_Population_Data.csv'
df = pd.read_csv(url)
df_t =pd.read_csv(url)

**Transformation** (more at the end)

In [ ]:
rows, columns = df_t.shape
#removing rows with null values
df_t.dropna(inplace=True)
print(f"The DataFrame has {rows} rows and {columns} columns.")

The DataFrame has 150482 rows and 17 columns.


In [ ]:
# Removing Duplicates
df.drop_duplicates(inplace=True)

In [ ]:
#The data type of each column
print(df.dtypes)

VIN (1-10)                                            object
County                                                object
City                                                  object
State                                                 object
Postal Code                                          float64
Model Year                                             int64
Make                                                  object
Model                                                 object
Electric Vehicle Type                                 object
Clean Alternative Fuel Vehicle (CAFV) Eligibility     object
Electric Range                                         int64
Base MSRP                                              int64
Legislative District                                 float64
DOL Vehicle ID                                         int64
Vehicle Location                                      object
Electric Utility                                      object
2020 Census Tract       

**CREATING DIMENSIONS AND FACT TABLE(Loading & Generating Surrogate Keys)**

VIN Dimension table

In [ ]:
# Create the VIN Dimension table
# tekes 1:30sec to run
create_table_vin_dim = """
CREATE TABLE IF NOT EXISTS VIN_Dim (
    VIN_Key SERIAL PRIMARY KEY,
    VIN VARCHAR(17) UNIQUE
)
"""
cursor.execute(create_table_vin_dim)
connection.commit()

# Prepare VIN DataFrame
vin_df = df[['VIN (1-10)']].drop_duplicates().reset_index(drop=True)
vin_df.rename(columns={'VIN (1-10)': 'VIN'}, inplace=True)

# Insert VIN data into VIN_Dim
insert_query_vin_dim = """
INSERT INTO VIN_Dim (VIN)
VALUES (%s)
ON CONFLICT (VIN) DO NOTHING
"""

for _, row in vin_df.iterrows():
    try:
        cursor.execute(insert_query_vin_dim, (row['VIN'],))
    except Exception as e:
        print(f"Error inserting VIN {row['VIN']}: {e}")
        connection.rollback()

connection.commit()


Location Dimension table

In [ ]:
# Create the Location Dimension table
# tekes 1:30sec to run
create_table_location_dim = """
CREATE TABLE IF NOT EXISTS Location_Dim (
    Location_Key SERIAL PRIMARY KEY,
    County VARCHAR(255),
    City VARCHAR(255),
    State CHAR(2),
    Postal_Code VARCHAR(20),
    Legislative_District VARCHAR(10),
    Vehicle_Location VARCHAR(255),
    Electric_Utility VARCHAR(255)
)
"""
cursor.execute(create_table_location_dim)
connection.commit()

# Prepare the Location DataFrame
location_df = df[['County', 'City', 'State', 'Postal Code', 'Legislative District', 'Vehicle Location', 'Electric Utility']].reset_index(drop=True)
location_df.columns = [column.replace(' ', '_') for column in location_df.columns]

# Insert data into Location_Dim
insert_query_location_dim = """
INSERT INTO Location_Dim (County, City, State, Postal_Code, Legislative_District, Vehicle_Location, Electric_Utility)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for _, row in location_df.iterrows():
    try:
        cursor.execute(insert_query_location_dim, (
            row['County'], row['City'], row['State'], row['Postal_Code'],
            row['Legislative_District'], row['Vehicle_Location'],
            row['Electric_Utility']
        ))
    except Exception as e:
        print(f"Error inserting location data: {e}")
        connection.rollback()

connection.commit()

Model Dimension table

In [ ]:
# Create the Model Dimension table
# tekes 1:30sec to run
df['Model Year'] = df['Model Year'].astype(int) # Convert 'Model Year' to integer

create_table_model_dim = """
CREATE TABLE IF NOT EXISTS Model_Dim (
    Model_Key SERIAL PRIMARY KEY,
    Make VARCHAR(255),
    Model VARCHAR(255),
    Model_Year INT
)
"""
cursor.execute(create_table_model_dim)
connection.commit()


model_df = df[['Make', 'Model', 'Model Year']].reset_index(drop=True)
model_df.columns = [column.replace(' ', '_') for column in model_df.columns]

# Insert data into Model_Dim
insert_query_model_dim = """
INSERT INTO Model_Dim (Make, Model, Model_Year)
VALUES (%s, %s, %s)
"""

for _, row in model_df.iterrows():
    try:
        cursor.execute(insert_query_model_dim, (
            row['Make'], row['Model'], row['Model_Year']
        ))
    except Exception as e:
        print(f"Error inserting model data: {e}")
        connection.rollback()

connection.commit()

Electric Vehicle Type Dimension table

In [ ]:
# Create the Electric Vehicle Type Dimension table
# tekes 1:30sec to run

df.rename(columns={
    'Clean Alternative Fuel Vehicle (CAFV) Eligibility': 'CAFV_Eligibility'
}, inplace=True)

create_table_ev_type_dim = """
CREATE TABLE IF NOT EXISTS EV_Type_Dim (
    Type_Key SERIAL PRIMARY KEY,
    Electric_Vehicle_Type VARCHAR(50),
    CAFV_Eligibility VARCHAR(255)
)
"""
cursor.execute(create_table_ev_type_dim)
connection.commit()

# Prepare the Electric Vehicle Type DataFrame
ev_type_df = df[['Electric Vehicle Type', 'CAFV_Eligibility']].reset_index(drop=True)
ev_type_df.columns = [column.replace(' ', '_').replace('(', '').replace(')', '') for column in ev_type_df.columns]  # Remove spaces and parentheses

# Insert data into EV_Type_Dim
insert_query_ev_type_dim = """
INSERT INTO EV_Type_Dim (Electric_Vehicle_Type, CAFV_Eligibility)
VALUES (%s, %s)
"""

for _, row in ev_type_df.iterrows():
    try:
        cursor.execute(insert_query_ev_type_dim, (
            row['Electric_Vehicle_Type'], row['CAFV_Eligibility']
        ))
    except Exception as e:
        print(f"Error inserting EV type data: {e}")
        connection.rollback()

connection.commit()


Fact Table and populating it

In [ ]:
# Create the Vehicle_Facts table
# tekes 1:30sec to run
create_fact_table_sql = """
CREATE TABLE IF NOT EXISTS Vehicle_Facts (
    FactID SERIAL PRIMARY KEY,
    VIN_Key INT,
    Location_Key INT,
    Model_Key INT,
    Type_Key INT,
    Electric_Range INT,
    Base_MSRP NUMERIC,
    FOREIGN KEY (VIN_Key) REFERENCES VIN_Dim(VIN_Key),
    FOREIGN KEY (Location_Key) REFERENCES Location_Dim(Location_Key),
    FOREIGN KEY (Model_Key) REFERENCES Model_Dim(Model_Key),
    FOREIGN KEY (Type_Key) REFERENCES EV_Type_Dim(Type_Key)
);
"""
cursor.execute(create_fact_table_sql)
connection.commit()

#Here we used these function instead of join since this is faster! But using join is also possible
def get_key_from_vin_dim(vin):
    cursor.execute("SELECT VIN_Key FROM VIN_Dim WHERE VIN = %s", (vin,))
    result = cursor.fetchone()
    return result[0] if result else None

def get_key_from_location_dim(county, city, state):
    cursor.execute("""
        SELECT Location_Key FROM Location_Dim WHERE County = %s AND City = %s AND State = %s
    """, (county, city, state))
    result = cursor.fetchone()
    return result[0] if result else None

def get_key_from_model_dim(make, model):
    cursor.execute("SELECT Model_Key FROM Model_Dim WHERE Make = %s AND Model = %s", (make, model))
    result = cursor.fetchone()
    return result[0] if result else None

def get_key_from_ev_type_dim(ev_type):
    cursor.execute("SELECT Type_Key FROM EV_Type_Dim WHERE Electric_Vehicle_Type = %s", (ev_type,))
    result = cursor.fetchone()
    return result[0] if result else None



insert_fact_query = """
INSERT INTO Vehicle_Facts (
    VIN_Key, Location_Key, Model_Key, Type_Key, Electric_Range, Base_MSRP
) VALUES (%s, %s, %s, %s, %s, %s)
"""

# Insert only the first 100 rows into the Vehicle_Facts table to test since the whole thing takes long
counter = 0
for _, row in df.iterrows():
    if counter >= 100:
        break  # Stop the loop after inserting 100 rows

    try:
        # Get the keys from the dimension tables using the provided functions
        vin_key = get_key_from_vin_dim(row['VIN (1-10)'])
        location_key = get_key_from_location_dim(row['County'], row['City'], row['State'])
        model_key = get_key_from_model_dim(row['Make'], row['Model'])
        type_key = get_key_from_ev_type_dim(row['Electric Vehicle Type'])

        # If any key is None (not found), skip the insert for this row
        if None in (vin_key, location_key, model_key, type_key):
            continue

        # Execute the insert query
        cursor.execute(insert_fact_query, (
            vin_key, location_key, model_key, type_key,
            row['Electric Range'], row['Base MSRP']
        ))

        counter += 1  # Increment the counter after a successful insert
    except Exception as e:
        print(f"Error inserting data on row {counter}: {e}")
        connection.rollback()  # Rollback the transaction so you can continue with other inserts

connection.commit()



**TRANSFORMATION(CONTINUED)**

**Icebergs**

In [ ]:
# Cars sold only in Seattle
seattle_cars = df[df['City'] == 'Seattle'].head(5)

# Top 5 cities with most EVs
top_5_ev_cities = df['City'].value_counts().head(5)


# Display the results
print("Cars sold only in Seattle:\n", seattle_cars)
print("\nTop 5 cities with most EVs:\n", top_5_ev_cities)

Cars sold only in Seattle:
     VIN (1-10) County     City State  Postal Code  Model Year     Make  \
0   KM8K33AGXL   King  Seattle    WA      98103.0        2020  HYUNDAI   
16  WA1VABGE4K   King  Seattle    WA      98112.0        2019     AUDI   
17  1N4AZ0CP6F   King  Seattle    WA      98125.0        2015   NISSAN   
23  2T3YL4DV5E   King  Seattle    WA      98108.0        2014   TOYOTA   
33  1N4AZ0CP9F   King  Seattle    WA      98109.0        2015   NISSAN   

     Model           Electric Vehicle Type  \
0     KONA  Battery Electric Vehicle (BEV)   
16  E-TRON  Battery Electric Vehicle (BEV)   
17    LEAF  Battery Electric Vehicle (BEV)   
23    RAV4  Battery Electric Vehicle (BEV)   
33    LEAF  Battery Electric Vehicle (BEV)   

                           CAFV_Eligibility  Electric Range  Base MSRP  \
0   Clean Alternative Fuel Vehicle Eligible             258          0   
16  Clean Alternative Fuel Vehicle Eligible             204          0   
17  Clean Alternative Fuel V

**data discretization**

In [ ]:
#data discretization (i.e., converting continuous data into discrete data by grouping it into bins or categories),

# Define bins for categorization
bins = [0, 100, 200, 300, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

# Discretize the 'ElectricRange' column
df['Electric Range'] = pd.cut(df['Electric Range'], bins=bins, labels=labels, include_lowest=True)

# Display the first few rows to verify the new column
print(df.head())

   VIN (1-10)    County      City State  Postal Code  Model Year     Make  \
0  KM8K33AGXL      King   Seattle    WA      98103.0        2020  HYUNDAI   
1  1C4RJYB61N      King   Bothell    WA      98011.0        2022     JEEP   
2  1C4RJYD61P    Yakima    Yakima    WA      98908.0        2023     JEEP   
3  5YJ3E1EA7J      King  Kirkland    WA      98034.0        2018    TESLA   
4  WBY7Z8C5XJ  Thurston   Olympia    WA      98501.0        2018      BMW   

            Model                   Electric Vehicle Type  \
0            KONA          Battery Electric Vehicle (BEV)   
1  GRAND CHEROKEE  Plug-in Hybrid Electric Vehicle (PHEV)   
2  GRAND CHEROKEE  Plug-in Hybrid Electric Vehicle (PHEV)   
3         MODEL 3          Battery Electric Vehicle (BEV)   
4              I3  Plug-in Hybrid Electric Vehicle (PHEV)   

                          CAFV_Eligibility Electric Range  Base MSRP  \
0  Clean Alternative Fuel Vehicle Eligible           High          0   
1    Not eligible due to l

**Feature engineering**

In [ ]:
# Sample DataFrame column setup
df = pd.DataFrame({
    'ElectricRange': [258, 25, 215, 97, 266],
    'BaseMSRP': [37000, 35000, 49000, 44000, 68000]
})

# Feature Engineering: Fuel Efficiency Category
# Calculate range per dollar (ElectricRange/BaseMSRP)
df['RangePerDollar'] = df['ElectricRange'] / df['BaseMSRP']

# Define bins for categorization of RangePerDollar
bins = [0, np.percentile(df['RangePerDollar'], 33), np.percentile(df['RangePerDollar'], 66), float('inf')]
labels = ['Low Efficiency', 'Medium Efficiency', 'High Efficiency']

# Categorize RangePerDollar into Efficiency Categories
df['EfficiencyCategory'] = pd.cut(df['RangePerDollar'], bins=bins, labels=labels, include_lowest=True)

**Creating special table(aggregate)**

In [ ]:
# Filter the DataFrame for only Tesla vehicles
tesla_df = df2[df2['Make'] == 'TESLA']
tesla_df[0:5]

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
3,5YJ3E1EA7J,King,Kirkland,WA,98034.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,45.0,104714466,POINT (-122.209285 47.71124),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
5,5YJ3E1EAXL,Snohomish,Marysville,WA,98271.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266,0,38.0,124595523,POINT (-122.1713847 48.10433),PUGET SOUND ENERGY INC,5.306194e+10
7,5YJYGDEE3L,King,Woodinville,WA,98072.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,45.0,124760555,POINT (-122.151665 47.75855),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
8,5YJ3E1EA1J,Island,Coupeville,WA,98239.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,10.0,125048003,POINT (-122.6880708 48.2179983),PUGET SOUND ENERGY INC,5.302997e+10
9,7SAYGDEF0P,King,Bellevue,WA,98004.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,48.0,240416207,POINT (-122.201905 47.61385),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10


In [ ]:
create_table_query = """
CREATE TABLE IF NOT EXISTS tesla_vehicles (
    VIN VARCHAR(20) PRIMARY KEY,
    County VARCHAR(255),
    City VARCHAR(255),
    State VARCHAR(2),
    PostalCode VARCHAR(10),
    ModelYear INT,
    Make VARCHAR(50),
    Model VARCHAR(50),
    ElectricVehicleType VARCHAR(50),
    CAFVEligibility VARCHAR(255),
    ElectricRange INT,
    BaseMSRP INT,
    LegislativeDistrict INT,
    DOLVehicleID BIGINT,
    VehicleLocation VARCHAR(255),
    ElectricUtility VARCHAR(255),
    CensusTract BIGINT
)
"""

cursor.execute(create_table_query)
connection.commit()

In [ ]:
#Inserting tesla vehicles table
#Inserting tesla vehicles table
insert_query = """
INSERT INTO tesla_vehicles (
    VIN, County, City, State, PostalCode, ModelYear, Make, Model,
    ElectricVehicleType, CAFVEligibility, ElectricRange, BaseMSRP,
    LegislativeDistrict, DOLVehicleID, VehicleLocation, ElectricUtility, CensusTract
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (VIN) DO NOTHING  -- Prevents duplicate entries
"""

for _, row in tesla_df.iterrows():
    try:
        # Removed the .year conversion since 'Model Year' is already an integer
        cursor.execute(insert_query, (
            row['VIN (1-10)'], row['County'], row['City'], row['State'], str(row['Postal Code']),
            row['Model Year'],  # Assuming this is already an integer
            row['Make'], row['Model'], row['Electric Vehicle Type'], row['Clean Alternative Fuel Vehicle (CAFV) Eligibility'],
            row['Electric Range'], row['Base MSRP'], row['Legislative District'], row['DOL Vehicle ID'],
            row['Vehicle Location'], row['Electric Utility'], row['2020 Census Tract']
        ))
    except Exception as e:
        #print("Error inserting data:", e)
        connection.rollback()  # Rollback the transaction

connection.commit()


In [ ]:
#Pulling from our databse
cursor.execute("SELECT * FROM tesla_vehicles LIMIT 5")
for row in cursor.fetchall():
    print(row)

('5YJSA1E21J', 'Kitsap', 'Port Orchard', 'WA', '98367.0', 1970, 'TESLA', 'MODEL S', 'Battery Electric Vehicle (BEV)', 'Clean Alternative Fuel Vehicle Eligible', 249, 0, 26, 220523628, 'POINT (-122.6851642 47.506453)', 'PUGET SOUND ENERGY INC', 53035092102)
('5YJ3E1EA3K', 'Snohomish', 'Marysville', 'WA', '98270.0', 1970, 'TESLA', 'MODEL 3', 'Battery Electric Vehicle (BEV)', 'Clean Alternative Fuel Vehicle Eligible', 220, 0, 38, 478795530, 'POINT (-122.17673 48.05542)', 'PUGET SOUND ENERGY INC', 53061052906)
('5YJXCDE47M', 'Island', 'Clinton', 'WA', '98236.0', 1970, 'TESLA', 'MODEL X', 'Battery Electric Vehicle (BEV)', 'Eligibility unknown as battery range has not been researched', 0, 0, 10, 137469244, 'POINT (-122.359364 47.9796552)', 'PUGET SOUND ENERGY INC', 53029972000)
('7SAYGDEF0P', 'King', 'Renton', 'WA', '98058.0', 1970, 'TESLA', 'MODEL Y', 'Battery Electric Vehicle (BEV)', 'Eligibility unknown as battery range has not been researched', 0, 0, 5, 221555400, 'POINT (-122.1298876 47

**Creating a table for top 5 cars range wise**

In [ ]:
unique_make_model = df2.sort_values(by='Electric Range', ascending=False) \
                      .drop_duplicates(subset=['Make', 'Model'])


top_5_unique_make_model = unique_make_model.head(5)

print(top_5_unique_make_model[['Make', 'Model', 'Electric Range']])


             Make    Model  Electric Range
47882       TESLA  MODEL S             337
51283       TESLA  MODEL 3             322
128004      TESLA  MODEL X             293
69064       TESLA  MODEL Y             291
139885  CHEVROLET  BOLT EV             259


In [ ]:
#Creating a table for top_5_range_cars
create_table_query2 = """
CREATE TABLE IF NOT EXISTS top_5_range (
    ID SERIAL PRIMARY KEY,
    Make VARCHAR(50),
    Model VARCHAR(50),
    ElectricRange INT
)
"""

cursor.execute(create_table_query2)
connection.commit()

# Inserting Top 5 cars with the highest range
insert_query_top_5_range = """
INSERT INTO top_5_range (Make, Model, ElectricRange)
VALUES (%s, %s, %s)
"""

for _, row in top_5_unique_make_model.iterrows():
    try:
        cursor.execute(insert_query_top_5_range, (row['Make'], row['Model'], row['Electric Range']))
    except Exception as e:
        print(f"Error inserting data for model {row['Model']}: {e}")
        connection.rollback()

connection.commit()

In [ ]:
#Pulling from our databse
cursor.execute("SELECT * FROM top_5_range LIMIT 5")
for row in cursor.fetchall():
    print(row)

(1, 'TESLA', 'MODEL S', 337)
(2, 'TESLA', 'MODEL 3', 322)
(3, 'TESLA', 'MODEL X', 293)
(4, 'TESLA', 'MODEL Y', 291)
(5, 'CHEVROLET', 'BOLT EV', 259)
